# Cergy-Pontoise

Test to create model of subsurface below Cergy from pikcs of hand-drawn map of the subsurface

In [ ]:
import os

# Importing GemPy
import gempy as gp

# VTK export
import pyevtk

# Importing aux libraries
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# Import pandas and geopandas
import pandas as pd
import geopandas as pgd

# Projections for coordinates
from pyproj import Proj
import rasterio as rio

# To download DEM
import elevation

Import the text file of my picks using SKUA-GOCAD (Negative is up in GOCAD and down in GemPy, so need to flip Z)

In [ ]:
file = '/work/armitagj/skua/TIGA/horizons_test.txt'
colnames = ['horizon', 'X', 'Y', 'Z']
df = pd.read_csv(file, names=colnames, header=None)
df['Z'] = -df['Z']
df.describe()

Need to translate the arbitrary X, Y, Z coordinates into topography and elevation. The idea is to use some geographical markers to convert from X, Y to decimal Lat, Long. For elevation, the map is relative to N.G.F. (https://en.wikipedia.org/wiki/General_levelling_of_France) so lets take that as sea-level for now.

In [ ]:
file = '/work/armitagj/skua/TIGA/georeference_test.txt'
colnames = ['X', 'Y', 'Lat', 'Long']
refdf = pd.read_csv(file, names=colnames, header=None)

In [ ]:
fig, axs = plt.subplots(2)
fig.suptitle('X, Y to Lat, Long')
axs[0].scatter(refdf.X, refdf.Long)
axs[1].scatter(refdf.Y, refdf.Lat)
plt.show()

In [ ]:
mxlong, cxlong = np.linalg.lstsq(np.vstack([refdf.X, np.ones(len(refdf))]).T, refdf.Long, rcond=None)[0]
mylat, cylat = np.linalg.lstsq(np.vstack([refdf.Y, np.ones(len(refdf))]).T, refdf.Lat, rcond=None)[0]

In [ ]:
x = np.array([6500, 10500])
y = np.array([20000, 29000])

fig, axs = plt.subplots(2)
fig.suptitle('X, Y to Lat, Long')
axs[0].scatter(refdf.X, refdf.Long)
axs[0].plot(x, x*mxlong + cxlong, 'r')
axs[1].scatter(refdf.Y, refdf.Lat)
axs[1].plot(y, y*mylat + cylat, 'r')
plt.show()

Define the model for GemPy with the extent and the resolution

Use linear regression to convert the arbitrary X, Y to lat, long

In [ ]:
df['Long'] = df['X']*mxlong + cxlong
df['Lat'] = df['Y']*mylat + cylat

In [ ]:
myProj = Proj("+proj=utm +zone=31N, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
df['UTMx'], df['UTMy'] = myProj(df['Long'].values, df['Lat'].values)
print(df.UTMx.min())
print(df.UTMx.max())
print(df.UTMy.min())
print(df.UTMy.max())

Use UTM Lat, Long data for model

In [ ]:
minx = df.UTMx.min()
maxx = df.UTMx.max()
miny = df.UTMy.min()
maxy = df.UTMy.max()
minz = df.Z.min()
maxz = df.Z.max()
resx = 100
resy = 100
resz = 50

geo_model = gp.create_model('cergy_pontoise')
geo_model = gp.init_data(geo_model,
                         extent= [minx, maxx, miny, maxy, minz, maxz], # of the regular grid
                         resolution=[resx, resy, resz])         # of the regular grid

In [ ]:
gp.set_interpolator(
    geo_model,
    compile_theano=True,
    theano_optimizer='fast_compile',
    gradient=False) # alternatives: 'fast_run' or
                                     # check http://deeplearning.net/software/theano/tutorial/modes.html 

In [ ]:
#p3d = gp.plot_3d(geo_model, plotter_type='background', notebook=False)

Create the surfaces from the dataframe. They need to be in stratigraphic order, youngest first.

In [ ]:
geo_model.add_surfaces(['Fontaine', 'Glaises', 'Marnes_m', 'Sables_M', 'Marno', 'Sables_de_B', 'Marnes_cail', 'Craie', 'Sables', 'Fausses_A', 'Basement'])

Read in the horizons from the dataframe and order them with IDs that are useful

In [ ]:
for index in range(len(df)):
    geo_model.add_surface_points(X=df.UTMx.iloc[index], Y=df.UTMy.iloc[index], Z=df.Z.iloc[index], surface=df.horizon.iloc[index])

Add orientation

In [ ]:
for index in range(len(df)):
    geo_model.add_orientations(X=df.UTMx.iloc[index], Y=df.UTMy.iloc[index], Z=df.Z.iloc[index], surface=df.horizon.iloc[index],
                           pole_vector= (0,0,1))


In [ ]:
#p3d.plot_data()

Compute the model

In [ ]:
# Compute model
#gp.compute_model(geo_model)

#p3d.plot_surfaces()
#p3d.plot_structured_grid(opacity=.2)

Get topography (there is a bug if my topo is the exact same size as the model, so added some padding to it)

In [ ]:
minlon = df.Long.min()-0.01
maxlon = df.Long.max()+0.01
minlat = df.Lat.min()-0.01
maxlat = df.Lat.max()+0.01

# Create a temporary file to store the DEM and go get it using elevation
dem_path = 'tmp.tif'
output = os.getcwd() + '/' + dem_path
elevation.clip(bounds=(minlon, minlat, maxlon, maxlat), output=output, product='SRTM1')

In [ ]:
from osgeo import gdal

gdal_data = gdal.Open(output)
data_array = gdal_data.ReadAsArray().astype(np.float)
plt.figure()
plt.imshow(data_array)
plt.colorbar()
plt.show

In [ ]:
with rio.open(output) as src:
    print(src.crs)

In [ ]:
dem_path = 'tmpUTM.tif'
topoDEM = os.getcwd() + '/' + dem_path
dst_crs = 'EPSG:32631'

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

with rio.open(output) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rio.open(topoDEM, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [ ]:
geo_model.set_topography(source='gdal', filepath=topoDEM)

In [ ]:
gp.plot_2d(geo_model, show_topography=True, section_names=['topography'], show_lith=False,
           show_boundaries=False,
           kwargs_topography={'cmap': 'gray', 'norm': None}
           )
plt.show()

Compute a new model

In [ ]:
# Compute model
gp.compute_model(geo_model, compute_mesh=False, set_solutions=True)

p3d_ = gp.plot_3d(geo_model, plotter_type='background', notebook=False)
p3d_.plot_surfaces()
#p3d_.plot_structured_grid(opacity=.2)

In [ ]:
g3d = gp.plot_3d(geo_model,
                 show_topography=True,
                 show_lith=False,
                 show_surfaces=False,
                 show_results=False,
                 ve=5,
                 plotter_type='background',
                 notebook=False)

In [ ]:
p3d_ = gp.plot_3d(geo_model,
                  plotter_type='background',
                  notebook=False,
                  ve=5)
p3d_.plot_surfaces()

Get the 3D volume plotted in the line above, to understand it better 

In [ ]:
xmin = geo_model.solutions.grid.regular_grid.extent[0]        #min coordinate value (left)
xmax = geo_model.solutions.grid.regular_grid.extent[1]        #max coordinate value (right)
xres = geo_model.solutions.grid.regular_grid.resolution[0]    #number of pixels
dx   = (xmax-xmin)/xres                       #pixel width
xvals = np.arange(xmin,xmax+dx,dx)            #calculate x coordinate values of the boundaries between cells

ymin = geo_model.solutions.grid.regular_grid.extent[2]
ymax = geo_model.solutions.grid.regular_grid.extent[3]
yres = geo_model.solutions.grid.regular_grid.resolution[1]
dy   = (ymax-ymin)/yres
yvals = np.arange(ymin,ymax+dy,dy)

zmin = geo_model.solutions.grid.regular_grid.extent[4]
zmax = geo_model.solutions.grid.regular_grid.extent[5]
zres = geo_model.solutions.grid.regular_grid.resolution[2]
dz   = (zmax-zmin)/zres
zvals = np.arange(zmin,zmax+dz,dz)

In [ ]:
print(np.shape(xvals))

In [ ]:
g = geo_model.solutions.lith_block.copy()            #make a copy to avoid messing up original
g = np.reshape(g, (xres,yres,zres))

Save x, y, z, and g for plotting within ipyleaflet in another notebook...

In [ ]:
np.save('x-coord.npy', xvals)
np.save('y-coord.npy', yvals)
np.save('z-coord.npy', zvals)
np.save('lith_block.npy', g)

save lith_block at topography to vtk (https://github.com/cgre-aachen/gempy/issues/159)

In [ ]:
path = 'tiga-model' # no extension
pyevtk.hl.gridToVTK(path, xvals, yvals, zvals, cellData={'data': g}) #export to VTK

try save to pickle to get the state into a simple notebook

In [ ]:
gp.save_model_to_pickle(geo_model, 'checkpoint1')